* 월별 총 결제수 / 총 결제액 / 평균 결재액 뽑기 - payment table

In [1]:
import MySQLdb
import pandas as pd

db = MySQLdb.connect(
    "db.fastcamp.us",
    "root",
    "dkstncks",
    "sakila",
    charset='utf8',
)

** 1. SQL**

In [2]:
SQL_QUERY = """
    SELECT * FROM payment;
"""

payment_df = pd.read_sql(SQL_QUERY, db)

In [5]:
payment_df.tail()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
16044,16045,599,1,14599,4.99,2005-08-21 17:43:42,2006-02-15 22:24:12
16045,16046,599,1,14719,1.99,2005-08-21 21:41:57,2006-02-15 22:24:12
16046,16047,599,2,15590,8.99,2005-08-23 06:09:44,2006-02-15 22:24:12
16047,16048,599,2,15719,2.99,2005-08-23 11:08:46,2006-02-15 22:24:13
16048,16049,599,2,15725,2.99,2005-08-23 11:25:00,2006-02-15 22:24:13


In [10]:
# 항상 FROM table 먼저하고 어떤것을 빼올지 차례차례 적어나가는게 중요
# 만약 average를 구할때도 차례차례구한다음 COUNT와 SUM을 #으로 지워주면 됨
SQL_QUERY = """
    SELECT
        LEFT(payment.payment_date, 7) "Month",
        COUNT(payment.payment_id) "Total Payment Count",
        SUM(payment.amount) "Total Payment Amount",
        (SUM(payment.amount) / COUNT(payment.payment_id)) "Amount Per Payment"
    FROM
        payment
    GROUP BY 1
    ;
"""

df = pd.read_sql(SQL_QUERY, db)
df.head()

,Month,Total Payment Count,Total Payment Amount,Amount Per Payment
0,2005-05,1157,4824.43,4.169775
1,2005-06,2312,9631.88,4.166038
2,2005-07,6711,28373.89,4.227968
3,2005-08,5687,24072.13,4.232835
4,2006-02,182,514.18,2.825165


In [11]:
6711 * 4.227968

28373.893247999997

** 2. pandas **

In [37]:
SQL_QUERY = """
    SELECT *
    FROM payment
;
"""

payment_df = pd.read_sql(SQL_QUERY, db)
payment_df.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
2,3,1,1,1185,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30
3,4,1,2,1422,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30
4,5,1,2,1476,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30


In [38]:
payment_df['month'] = payment_df['payment_date'].apply(
lambda x: str(x)[:7]
)

In [39]:
payment_df.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update,month
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30,2005-05
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30,2005-05
2,3,1,1,1185,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30,2005-06
3,4,1,2,1422,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30,2005-06
4,5,1,2,1476,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30,2005-06


count/sum/average

In [40]:
count_series = payment_df.groupby('month').size()
count_series

month
2005-05    1157
2005-06    2312
2005-07    6711
2005-08    5687
2006-02     182
dtype: int64

* aggregate쓰기
    * 집합, 합치다 : 데이터를 합치는 기능

In [43]:
import numpy as np

In [47]:
payment_df.groupby('month').agg({
        'payment_id' : [np.size,],   # COUNT
        'amount' : [np.sum, np.mean],   # SUM, MEAN
               
    })

payment_id    amount          
              size       sum      mean
month                                 
2005-05       1157   4824.43  4.169775
2005-06       2312   9631.88  4.166038
2005-07       6711  28373.89  4.227968
2005-08       5687  24072.13  4.232835
2006-02        182    514.18  2.825165

In [48]:
a = payment_df.groupby('month')

In [49]:
a.agg?

In [51]:
payment_df.groupby('month').sum()

,payment_id,customer_id,staff_id,rental_id,amount
month,,,,,
2005-05,9125123,338396,1697,669582,4824.43
2005-06,18378719,681326,3460,5346565,9631.88
2005-07,54209057,2008040,10076,45786570,28373.89
2005-08,45674417,1689425,8539,74459462,24072.13
2006-02,1405909,51977,269,2496881,514.18
